In [76]:
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import json
import random
import pickle

In [77]:
intents = json.load(open("intents.json"))

In [3]:
words = []
classes = []
docs = []
ignore_tokens = ['?', '!', '.', ',']

In [4]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        docs.append((word_list, intent["tag"]))
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [8]:
lematizer = WordNetLemmatizer()

In [6]:
words = [lematizer.lemmatize(word.lower()) for word in words if word not in ignore_tokens]

In [7]:
words = sorted(set(words))

In [8]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

In [9]:
training = []
output_empty = [0] * len(classes)

In [10]:
for doc in docs:
    bag = []
    word_patters = doc[0]
    word_patters = [lematizer.lemmatize(word.lower()) for word in word_patters]
    for word in words:
        bag.append(1) if word in word_patters else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [11]:
random.shuffle(training)
training = np.array(training)

C:\Users\vedan\AppData\Local\Temp/ipykernel_18592/2739997722.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [12]:
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [78]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [14]:
model = Sequential([
    layers.Dense(128, input_shape=(len(train_x[0]), ), activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(train_y[0]), activation="softmax")
])
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["acc"]
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5888      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 14,469
Trainable params: 14,469
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_x, train_y, epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.6396 - acc: 0.1905
Epoch 2/100
1/1 [==============================] - 0s 5ms/step - loss: 1.6334 - acc: 0.1429
Epoch 3/100
1/1 [==============================] - 0s 6ms/step - loss: 1.6279 - acc: 0.2381
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 1.5772 - acc: 0.1905
Epoch 5/100
1/1 [==============================] - 0s 7ms/step - loss: 1.5967 - acc: 0.1429
Epoch 6/100
1/1 [==============================] - 0s 4ms/step - loss: 1.5519 - acc: 0.3810
Epoch 7/100
1/1 [==============================] - 0s 6ms/step - loss: 1.5247 - acc: 0.4762
Epoch 8/100
1/1 [==============================] - 0s 5ms/step - loss: 1.5434 - acc: 0.3333
Epoch 9/100
1/1 [==============================] - 0s 5ms/step - loss: 1.5022 - acc: 0.2857
Epoch 10/100
1/1 [==============================] - 0s 5ms/step - loss: 1.5059 - acc: 0.4286
Epoch 11/100
1/1 [==============================] - 0s 4ms/step - loss: 1.4826 -

In [4]:
model.save("bot_V1.model")

NameError: name 'model' is not defined

In [79]:
model = keras.models.load_model("bot_V1.model")
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

In [80]:
def preprocess(item):
    tokens = nltk.word_tokenize(item)
    tokens = [lematizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(sentence):
    sentence_words = preprocess(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    
    return np.array(bag)


In [81]:
def predict_class(sentence, tolerance=0.7):
    bow = bag_of_words(sentence)
    prediction = model(np.array([bow]))[0]
    max_value = max(prediction).numpy()
    res = np.where(prediction.numpy() == max_value)[0][0]
    if max_value >= tolerance:
        return res
    else:
        return -1

In [82]:
predict_class("Hello how are you")

0

In [83]:
from chatbot import Chatbot

In [97]:
class Chatbot:
    def __init__(self, intent_bot_path, ner_bot_path=None, words_path="words.pkl", classes_path="classes.pkl", intents_path="intents.json"):
        print("loading models")
        self.intent_classifier = keras.models.load_model(intent_bot_path)
        # load the ner bot 
        self.words = pickle.load(open(words_path, "rb"))
        self.classes = pickle.load(open(classes_path, "rb"))
        self.intents = json.load(open(intents_path, "r"))["intents"]
        self.lemmatizer = WordNetLemmatizer()
        print("models loaded")
    
    def preprocess(self, item):
        tokens = nltk.word_tokenize(item)
        tokens = [self.lemmatizer.lemmatize(word) for word in tokens]
        return tokens

    def bag_of_words(self, sentence):
        sentence_words = self.preprocess(sentence)
        bag = [0] * len(self.words)
        for w in sentence_words:
            for i, word in enumerate(self.words):
                if word == w:
                    bag[i] = 1
        
        return np.array(bag)

    def predict_intent(self, sentence, tolerance=0.5):
        bow = self.bag_of_words(sentence)
        prediction = self.intent_classifier(np.array([bow]))[0]
        max_value = max(prediction).numpy()
        res = np.where(prediction.numpy() == max_value)[0][0]
        return self.intents[res]["tag"]

In [98]:
bot = Chatbot("bot_V1.model")

loading models
models loaded


In [99]:
bot.predict_intent("I wish to buy something")

'shop'